## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-03-20-52-37 +0000,nyt,Venezuela Live Updates: Trump Says U.S. Will ‘...,https://www.nytimes.com/live/2026/01/03/world/...
1,2026-01-03-20-52-35 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/03/world/...
2,2026-01-03-20-52-04 +0000,nyt,"Delcy Rodríguez, Venezuela’s new leader, boast...",https://www.nytimes.com/2026/01/03/world/ameri...
3,2026-01-03-20-50-51 +0000,nyt,Trump administration declines to back Venezuel...,https://www.nytimes.com/live/2026/01/03/world/...
4,2026-01-03-20-48-04 +0000,nyt,"Before Nicolás Maduro, There Was Manuel Noriega",https://www.nytimes.com/2026/01/03/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,venezuela,40
8,maduro,38
3,trump,33
7,capture,16
14,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2026-01-03-20-52-37 +0000,nyt,Venezuela Live Updates: Trump Says U.S. Will ‘...,https://www.nytimes.com/live/2026/01/03/world/...,151
34,2026-01-03-18-42-03 +0000,latimes,Trump says U.S. will 'run' Venezuela after cap...,https://www.latimes.com/politics/story/2026-01...,137
39,2026-01-03-18-17-46 +0000,nypost,Trump brushes off concerns about legality of U...,https://nypost.com/2026/01/03/world-news/trump...,136
66,2026-01-03-15-57-33 +0000,nypost,Venezuela opposition leader Maria Corina Macha...,https://nypost.com/2026/01/03/world-news/venez...,128
14,2026-01-03-20-21-33 +0000,nyt,Venezuela’s Interim Leader Defies Trump and Ca...,https://www.nytimes.com/2026/01/03/world/ameri...,127


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,151,2026-01-03-20-52-37 +0000,nyt,Venezuela Live Updates: Trump Says U.S. Will ‘...,https://www.nytimes.com/live/2026/01/03/world/...
144,49,2026-01-03-03-00-00 +0000,wsj,The massive welfare-fraud scandal in Minnesota...,https://www.wsj.com/politics/policy/minnesota-...
78,40,2026-01-03-14-53-38 +0000,bbc,Swiss open criminal case against managers of s...,https://www.bbc.com/news/articles/cjdjl303zxlo...
125,40,2026-01-03-07-05-24 +0000,nypost,Florida boy Johnathan Boley found dead after h...,https://nypost.com/2026/01/03/us-news/florida-...
45,35,2026-01-03-17-50-54 +0000,nypost,"Venezuelan dictator appears blindfolded, handc...",https://nypost.com/2026/01/03/world-news/venez...
66,30,2026-01-03-15-57-33 +0000,nypost,Venezuela opposition leader Maria Corina Macha...,https://nypost.com/2026/01/03/world-news/venez...
132,26,2026-01-03-05-09-32 +0000,nypost,DOJ official warns Seattle mayor to leave jour...,https://nypost.com/2026/01/03/us-news/doj-offi...
31,25,2026-01-03-19-04-15 +0000,nypost,Inside Operation Absolute Resolve: How US forc...,https://nypost.com/2026/01/03/world-news/how-u...
89,20,2026-01-03-13-49-12 +0000,nypost,Staten Island seeing red: Forgotten boro more ...,https://nypost.com/2026/01/03/us-news/staten-i...
48,19,2026-01-03-17-39-00 +0000,wsj,U.S. Strikes in Venezuela Disrupt Air Traffic ...,https://www.wsj.com/business/airlines/u-s-stri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
